In [1]:
from rdkit import Chem
import torch
torch.backends.cuda.matmul.allow_tf32 = False
torch.backends.cudnn.allow_tf32 = False

In [2]:
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
print(device)
in_path = 'example.sdf'

aimnet2 = torch.jit.load('/home/jack/aimnet2_lab/models/aimnet2_wb97m-d3_ens.jpt', map_location=device)

cuda:2


In [3]:
def sdf2aimnet_input(sdf: str, device=torch.device('cpu')) -> dict:
    """Converts sdf to aimnet input, assuming the sdf has only 1 conformer."""
    mol = next(Chem.SDMolSupplier(sdf, removeHs=False))
    conf = mol.GetConformer()
    coord = torch.tensor(conf.GetPositions(), device=device).unsqueeze(0)
    numbers = torch.tensor([atom.GetAtomicNum() for atom in mol.GetAtoms()], device=device).unsqueeze(0)
    charge = torch.tensor([Chem.GetFormalCharge(mol)], device=device, dtype=torch.float)
    return dict(coord=coord, numbers=numbers, charge=charge)


dct = sdf2aimnet_input(in_path, device=device)
dct['coord'].requires_grad_(True)
aimnet2_out = aimnet2(dct)
print('aimnet2 energy: ', aimnet2_out['energy'])  # there is no gradient for energy

aimnet2 energy:  tensor([-10518.2732], device='cuda:2', dtype=torch.float64)


In [4]:
for key, val in aimnet2_out.items():
    print(key, val.shape)

coord torch.Size([1, 20, 3])
numbers torch.Size([1, 20])
charge torch.Size([1])
charges torch.Size([1, 20])
charges_std torch.Size([1, 20])
energy torch.Size([1])
energy_std torch.Size([1])
forces torch.Size([1, 20, 3])
forces_std torch.Size([1, 20, 3])


In [5]:
print('aimnet2 energy: ', aimnet2_out['energy'])

aimnet2 energy:  tensor([-10518.2732], device='cuda:2', dtype=torch.float64)


In [ ]:
forces = -torch.autograd.grad(aimnet2_out['energy'], dct['coord'], create_graph=True)[0]
print(forces)